In [48]:
import cv2
from tensorflow import keras
import tensorflow as tf
from PIL import Image
import numpy as np


In [50]:
#Load data stuff and model
labels_names =['Deer','Human','Unknown']
data_train = np.load('data.npy')
labels_train = np.load('labels.npy')
labels_train = labels_train.astype(int)
class_labels, _ = np.unique(labels_train, return_counts=True)

model = keras.models.load_model('retrained_model_ResNet50_1.h5')

In [64]:
#Example with random deer image
image_path = 'test_validation_images/deer_3.png'
image = Image.open(image_path).convert('RGB')


image_array = np.array(image)

img_height = 224
img_width = 224

img = tf.image.resize(image_array, size=[img_height,img_width])
img =tf.expand_dims(img,0)

# Predictions
predictions = model.predict(img)
score = tf.nn.softmax(predictions[0])

print(
    "Class: {} - {} | {:.2f} %"
    .format(class_labels[np.argmax(score)], labels_names[class_labels[np.argmax(score)]-1] , 100 * np.max(score))
)


1/1 [==============================] - 0s 72ms/step
Class: 1 - Deer | 100.00 %


In [ ]:
img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = custom_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_labels[np.argmax(score)], 100 * np.max(score))
)